## SILVER TO GOLD LAYER

### Gold Layer - Holdings


In [1]:
## Import necessary libraries and utility functions
import pandas as pd

from ETLTools import GlobalPath, Portfolio

In [2]:
# Instantiate GlobalPath
tradehistory_silver_file_path = GlobalPath(
    "DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)
holdingshistory_gold_file_path = GlobalPath(
    "DATA/GOLD/Holdings/HoldingsHistory_data.csv"
)
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holdings/CurrentHoldings_data.csv"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [3]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Filter the DataFrame to include only rows where the segment is 'EQ' (Equity) or 'MF' (Mutual Funds)
df_trade_history = df_trade_history[
    df_trade_history["segment"].isin(["EQ", "MF"])
]

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(
    f"Read SILVER Layer trade history data from: {tradehistory_silver_file_path}"
)

Read SILVER Layer trade history data from: DATA/SILVER/TradeHistory/TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [4]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.astype(str).to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()
# expired_stocks example : df_trade_history["scrip_name"] == "NIFTY-PE-24650-18JUL2024"

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [5]:
# Create a DataFrame from the processed data
df_holding_records = pd.DataFrame(portfolio.get_current_holdings())

# Remove '_amount' suffix from column names for clarity
df_holding_records = df_holding_records.rename(
    columns={
        col: col.replace("open_", "") for col in df_holding_records.columns
    }
)
# Sort the DataFrame
df_holding_records = df_holding_records.sort_values(
    by=[
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
        "datetime",
    ]
)

# Round the values in to two decimal places
df_holding_records = df_holding_records.round(2)

# Reset index to ensure it starts from 0
df_holding_records = df_holding_records.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_holding_records = df_holding_records[
    [
        "scrip_name",
        "symbol",
        "exchange",
        "segment",
        "datetime",
        "side",
        "quantity",
        "price",
        "amount",
    ]
]

# Save the final DataFrame to a CSV file
df_holding_records.to_csv(current_holding_records_file_path, index=None)

print("GOLD Layer CSV file for Current Holdings successfully created at:")
print(current_holding_records_file_path)

# Display the DataFrame debugrmation
df_holding_records.info()

GOLD Layer CSV file for Current Holdings successfully created at:
DATA/GOLD/Holdings/CurrentHoldings_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   scrip_name  18 non-null     object        
 1   symbol      18 non-null     object        
 2   exchange    18 non-null     object        
 3   segment     18 non-null     object        
 4   datetime    18 non-null     datetime64[ns]
 5   side        18 non-null     object        
 6   quantity    18 non-null     float64       
 7   price       18 non-null     float64       
 8   amount      18 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.4+ KB


In [6]:
# Create a DataFrame from the processed data
df_holdings = pd.DataFrame(portfolio.get_holdings_history())

# Round the values in the columns to two decimal places
df_holdings = df_holdings.round(2)

# Extract date from datetime
df_holdings["date"] = df_holdings["datetime"].dt.date

# Find the index of the maximum datetime for each scrip_name and date
idx = df_holdings.groupby(["scrip_name", "date"])["datetime"].idxmax()

# Use the indices to filter the original DataFrame
df_holdings = df_holdings.loc[idx].reset_index(drop=True)

In [7]:
# Create a function to expand the date range for each stock
def expand_dates(stock_df):
    """
    Expands the date range for each stock to include all dates from the
    minimum date to the current date and forward fills the missing values.

    Parameters:
    stock_df (pd.DataFrame): DataFrame containing stock data for a single stock.

    Returns:
    pd.DataFrame: DataFrame with expanded date range and forward-filled values.
    """
    # Find the minimum date for this stock
    min_date = stock_df["date"].min()

    # Create a date range from the minimum date to today
    date_range = pd.date_range(start=min_date, end=pd.to_datetime("today"))

    # Reindex the stock_df to include the full date range
    stock_df = stock_df.set_index("date").reindex(date_range)

    # Forward fill the values to fill missing dates
    stock_df = stock_df.ffill().reset_index()

    # Rename the 'index' column to 'date'
    stock_df = stock_df.rename(columns={"index": "date"})

    return stock_df


# Apply the expand_dates function to each stock group
df_holdings = (
    df_holdings.groupby("scrip_name")
    .apply(expand_dates, include_groups=False)
    .reset_index()
)

In [8]:
# Load stock prices data from the specified CSV file path
df_stockprice = pd.read_csv(stockprice_silver_file_path)

# Convert the 'date' column in stock prices to datetime type for consistency
df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])

print(
    f"Loaded SILVER Layer stock price data from: {stockprice_silver_file_path}"
)

# Merge the expanded holdings data with the stock price data
df_holdings = pd.merge(
    df_holdings,
    df_stockprice,
    on=["date", "symbol"],
    how="left",
)

Loaded SILVER Layer stock price data from: DATA/SILVER/StockPrice/StockPrice_data.csv


In [9]:
# Calculate the OHLC value of holdings
col_names = ["open", "high", "low", "close"]
for col_name in col_names:
    df_holdings[f"{col_name}_price"] = df_holdings[col_name]
    df_holdings[f"{col_name}_amount"] = (
        df_holdings[col_name] * df_holdings["holding_quantity"]
    )

# Forward fill the values to fill missing dates
df_holdings = df_holdings.ffill()

# Filter out rows with zero holding quantity
df_holdings = df_holdings[df_holdings["holding_quantity"] != 0]

# Round the values in to two decimal places
df_holdings = df_holdings.round(2)

In [10]:
# Sort the DataFrame for organized output
df_holdings = df_holdings.sort_values(
    by=[
        "date",
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
    ]
)

# Reset index to ensure it starts from 0
df_holdings = df_holdings.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_holdings = df_holdings[
    [
        "date",
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
        "holding_quantity",
        "avg_price",
        "holding_amount",
        "open_price",
        "open_amount",
        "high_price",
        "high_amount",
        "low_price",
        "low_amount",
        "close_price",
        "close_amount",
    ]
]

# Save the final processed DataFrame to a new CSV file
df_holdings.to_csv(holdingshistory_gold_file_path, index=None)

print(
    f"GOLD Layer CSV file for Holdings successfully created at: {holdingshistory_gold_file_path}"
)

# Display debugrmation about the final DataFrame
df_holdings.info()

GOLD Layer CSV file for Holdings successfully created at: DATA/GOLD/Holdings/HoldingsHistory_data.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6073 entries, 0 to 6072
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              6073 non-null   datetime64[ns]
 1   segment           6073 non-null   object        
 2   exchange          6073 non-null   object        
 3   symbol            6073 non-null   object        
 4   scrip_name        6073 non-null   object        
 5   holding_quantity  6073 non-null   float64       
 6   avg_price         6073 non-null   float64       
 7   holding_amount    6073 non-null   float64       
 8   open_price        6073 non-null   float64       
 9   open_amount       6073 non-null   float64       
 10  high_price        6073 non-null   float64       
 11  high_amount       6073 non-null   float64       
 12  low_price         6073 non-null   float64       
 13  low_amount        6073 non-null   float64       
 14  close_price       6073 